In [1]:
import os
import sys
sys.path.insert(0, os.path.join(os.getcwd(), '..'))

from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv())

from src.crawl import Crawler

API_KEY = "" # Put your API key here

crawler = Crawler(api_key=API_KEY)

## Get data

### Get all categories and metrics

In [2]:
# Get all categories and metrics
all_categories_and_metrics = crawler.get_all_metrics()
categories = list(all_categories_and_metrics.keys())
print("There are %d categories in total" % len(categories))

print(categories)
for category in categories:
    for sub_category in all_categories_and_metrics[category]:
        for sub_sub_category in all_categories_and_metrics[category][sub_category]:
            print("There are %d metric(s) in subcategory %s of %s category" % (len(
                all_categories_and_metrics[category][sub_category][sub_sub_category]), sub_category, category))

There are 5 categories in total
['Tournament', 'Blockchain', 'Stocks', 'Forex', 'Economy']
There are 1 metric(s) in subcategory Return And Risk of Tournament category
There are 1 metric(s) in subcategory CEX of Blockchain category
There are 1 metric(s) in subcategory CEX of Blockchain category
There are 2 metric(s) in subcategory CEX of Blockchain category
There are 12 metric(s) in subcategory Stocks at exchange of Stocks category
There are 14 metric(s) in subcategory ETF & Indices of Stocks category
There are 23 metric(s) in subcategory ETF & Indices of Stocks category
There are 21 metric(s) in subcategory ETF & Indices of Stocks category
There are 22 metric(s) in subcategory ETF & Indices of Stocks category
There are 20 metric(s) in subcategory Raw currency pairs of Forex category
There are 51 metric(s) in subcategory FRED of Economy category
There are 31 metric(s) in subcategory FRED of Economy category
There are 17 metric(s) in subcategory FRED of Economy category


Example of supported metrics in Forex category

In [3]:
print("Some metrics in Forex category:")
all_categories_and_metrics['Forex']

Some metrics in Forex category:


{'Raw currency pairs': {'Raw currency pairs': [{'id': 'eurusd',
    'name': 'EURUSD'},
   {'id': 'usdcad', 'name': 'USDCAD'},
   {'id': 'usdchf', 'name': 'USDCHF'},
   {'id': 'gbpusd', 'name': 'GBPUSD'},
   {'id': 'nzdusd', 'name': 'NZDUSD'},
   {'id': 'audusd', 'name': 'AUDUSD'},
   {'id': 'usdjpy', 'name': 'USDJPY'},
   {'id': 'eurcad', 'name': 'EURCAD'},
   {'id': 'euraud', 'name': 'EURAUD'},
   {'id': 'eurjpy', 'name': 'EURJPY'},
   {'id': 'eurchf', 'name': 'EURCHF'},
   {'id': 'eurgbp', 'name': 'EURGBP'},
   {'id': 'audcad', 'name': 'AUDCAD'},
   {'id': 'gbpchf', 'name': 'GBPCHF'},
   {'id': 'gbpjpy', 'name': 'GBPJPY'},
   {'id': 'chfjpy', 'name': 'CHFJPY'},
   {'id': 'audjpy', 'name': 'AUDJPY'},
   {'id': 'audnzd', 'name': 'AUDNZD'},
   {'id': 'xptusd', 'name': 'XPTUSD'},
   {'id': 'xpdusd', 'name': 'XPDUSD'}]}}

### Download data

In [4]:
# Historical data (all)
crawler.download_historical_data(metric_id="usd_price", resolution="1d", symbol="BTC", location='./data')

# Latest data (last 24 hours)
print("Lastest data: ", crawler.get_lastest_data(metric_id="usd_price", resolution="1h", symbol="BTC"))

Folder './data/usd_price' already exists
Lastest data:          OPEN_TIME      OPEN     HIGH       LOW     CLOSE
0   1703768400000  42796.50  42998.0  42596.36  42799.59
1   1703772000000  42802.57  42893.0  42460.04  42671.80
2   1703775600000  42661.70  42775.0  42407.53  42451.96
3   1703779200000  42457.43  42628.0  42265.08  42461.38
4   1703782800000  42461.38  42704.0  42443.90  42560.70
5   1703786400000  42560.80  42701.0  42382.39  42484.59
6   1703790000000  42486.51  42806.0  42458.69  42613.77
7   1703793600000  42646.80  42671.0  42370.05  42515.29
8   1703797200000  42513.64  42649.0  42350.00  42464.89
9   1703800800000  42474.30  42789.2  42388.87  42655.00
10  1703804400000  42656.16  42892.8  42546.00  42592.73
11  1703808000000  42581.23  42739.9  42396.58  42434.92
12  1703811600000  42433.28  42585.0  42135.74  42375.20
13  1703815200000  42346.00  42649.0  42277.09  42602.30
14  1703818800000  42582.31  42870.0  42458.10  42708.15
15  1703822400000  42720.20  428

## Submit model

In [5]:
import time
from submit import Submission

s = Submission(tournament_search_id='return_and_risk_tournament', api_key=os.getenv('API_KEY'))

# Test data
symbol = 'BTC'
model_id = 'test_model_id'
cur_time = int(time.time())*1000

# Round down OPEN_TIME to round time (00:00 or 12:00 UTC), used for testing only
backtest_example_data = [{"OPEN_TIME": cur_time - cur_time%(12*3600000) - 12*100*3600000, "PREDICTION": 0.487618394396533}]

### Submission for back_test mode

In [6]:
submission_time = s.submit(is_backtest=True, data=backtest_example_data, symbol=symbol, model_id=model_id)
print("Example submission - Submission time:", submission_time)

Example submission - Submission time: 1703857293400


### Get all submissions

In [7]:
print("All backtest submissions:", s.get_submission_times(is_backtest=True, symbol=symbol))

All backtest submissions: {'back_test': {'BTC': {'test_input_7h_3_11': [1699633752935, 1699886805682], 'lgbm_model': [1700989856310, 1702432567687], 'test_model_id': [1703857293400]}, 'ETH': {}, 'BNB': {}, 'ORAI': {}}}


In [8]:
# Waiting for results
time.sleep(5)

### Get results

In [9]:
print("Get result:", s.get_result(is_backtest=True, symbol=symbol,
      model_id=model_id, submission_time=submission_time))

Get result: {'back_test': {'BTC': {'test_model_id': {'1703857293400': {'Header': ['Round', 'Movement Score', 'CORR15', 'MS15', 'Prediction', 'Label'], 'Data': [[1699531200000.0, 1.0, 0.0, 0.0, 0.487618394396533, 0.5233716868809766]]}}}}}


### Delete submission

In [10]:
res = s.delete_back_test_submission_times(symbol=symbol, model_id=model_id, submission_time=submission_time)
print(res)

Submission record is deleted successfully


### Get current round time

In [11]:
s.get_current_round_time()

{'Round time': 1703851200000, 'Round number': 110}